<a href="https://colab.research.google.com/github/Aditya-11/High_Speed_Solver/blob/main/CPU_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test CPU time for matrix multiplication !

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.220
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.44
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.220
BogoMIPS:            4400.44
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
%%writefile sam1.cpp

#include <bits/stdc++.h> 
#include <thread>
#include <chrono>

using namespace std; 
  
// maximum size of matrix 
#define MAX 10000 
  
// maximum number of threads 
#define MAX_THREAD 2

int step_i = 0 ; 

float * matA = nullptr ; 
float * matB = nullptr ; 
float * matC = nullptr ;


void* multi(void* arg) 
{ 
    int core = step_i++;

    // std::cout << "core no : " <<  core << std::endl;

    for (int i = core * MAX / MAX_THREAD; i < (core + 1) * MAX / MAX_THREAD; i++)  
    {
        for (int j = 0; j < MAX; j++)  
        { 
           float Pvalue = 0; 
           for (int k = 0; k < MAX ; ++k) {
            float Mdelement = matA[i*MAX + k];
            float Ndelement = matB[k*MAX + j];
            Pvalue += (Mdelement*Ndelement);
            }
            matC[j*MAX + i] = Pvalue ;
        }
    }
} 

int main () {
    matA = (float * )malloc (sizeof(float) * MAX * MAX) ; 
    matB = (float *)malloc (sizeof(float) * MAX * MAX) ; 
    matC = (float *)malloc (sizeof(float) * MAX * MAX) ; 

    for (int i = 0 ; i < MAX * MAX; i++ ) { 
            matA[i]= 1 ;
            matB[i] = 1 ;
    }

    pthread_t threads[MAX_THREAD]; 

    using clock = std::chrono::system_clock;
    using sec = std::chrono::duration<double>;

    const auto before = clock::now();

    for (int i = 0; i < MAX_THREAD; i++) { 
        int* p; 
        pthread_create(&threads[i], NULL, multi, (void*)(p)); 
    } 
  
    for (int i = 0; i < MAX_THREAD; i++)  pthread_join(threads[i], NULL);     

    const sec duration = clock::now() - before;
    std::cout <<  "time to multiply : " << duration.count() << std::endl << std::endl;

    std::cout << matC[1]<< " " << sizeof(matC) / sizeof(matC[0]) << std::endl ;

    delete [] matA ; 
    delete [] matB ;
    delete [] matC ; 
}

Writing sam1.cpp


In [ ]:
%%writefile sam1_vec.cpp

#include <bits/stdc++.h> 
#include <thread> 
#include <chrono> 

using namespace std; 
  
// maximum size of matrix 
#define MAX 10000 
  
// maximum number of threads 
#define MAX_THREAD 2 

int step_i = 0 ; 

float * matA = nullptr ; 
float * matB = nullptr ; 
float * matC = nullptr ;


void* multi(void* arg) 
{ 
    int core = step_i++;
    std::cout << "core no : " <<  core << std::endl;
    for (int i = core * MAX / MAX_THREAD; i < (core + 1) * MAX / MAX_THREAD; i++)  
    {
           float Pvalue = 0; 
           for (int k = 0; k < MAX ; ++k) 
           {
            float Mdelement = matA[i*MAX + k];
            float Ndelement = matB[k];
            Pvalue += (Mdelement*Ndelement);
           }
           matC[i] = Pvalue ;
          // printf("%d \t %f \n",i,Pvalue); 
    }
} 

int main () {
    
    matA = (float * )malloc (sizeof(float) * MAX * MAX) ; 
    matB = (float *)malloc (sizeof(float) * MAX) ; 
    matC = (float *)malloc (sizeof(float) * MAX) ; 

    for (int i = 0 ; i < MAX * MAX; i++ ) { 
        matA[i]= 1 ;
    }

    for (int i = 0 ; i< MAX; i++) { 
        matB[i] = 1; 
    }

    pthread_t threads[MAX_THREAD]; 

    using clock = std::chrono::system_clock;
    using sec = std::chrono::duration<double>;

    const auto before = clock::now();

    for (int i = 0; i < MAX_THREAD; i++) { 
        int* p; 
        pthread_create(&threads[i], NULL, multi, (void*)(p)); 
    } 
  
    for (int i = 0; i < MAX_THREAD; i++)  pthread_join(threads[i], NULL);     

    const sec duration = clock::now() - before;

    std::cout <<  "time to multiply : " << duration.count() << std::endl << std::endl;

    std::cout << matC[1]<< " " << matC[100] << " " << matC[MAX-1] <<  std::endl ;

    delete [] matA ; 
    delete [] matB ;
    delete [] matC ; 
}

Overwriting sam1_vec.cpp


In [ ]:
%%script bash

g++ sam1_vec.cpp -o test_vec -lpthread
./test_vec

core no : 0
core no : 1
time to multiply : 0.391942

10000 10000 10000


Using 4 threads to run the operation 

In [ ]:
%%script bash

g++ sam1.cpp -o test -lpthread
./test

Process is interrupted.


In [ ]:
%%script bash

g++ sam1.cpp -o test1 -lpthread
./test1

time to multiply : 13755.5

10000 10000
